## Demo Tracking pose and face

## Use input size :
- file cfg
- yolo3_weight.h5 when convert
- file train.py

In [8]:
import colorsys
import cv2
import dlib
import face_recognition
from keras import backend as K
from keras.utils import multi_gpu_model
from keras.layers import Input
from keras.models import load_model

import numpy as np
import os
from PIL import Image, ImageFont, ImageDraw
from timeit import default_timer as timer
from scipy.spatial import distance

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image

Using TensorFlow backend.


In [9]:
from imutils.video import WebcamVideoStream

In [10]:
# from threading import Thread
# import cv2
# import imutils
# class WebcamVideoStream:
#     def __init__(self, src=0):
#         self.stream = cv2.VideoCapture(src)
#         self.stream.set(3, 800)
#         self.stream.set(4, 600)
#         (self.grabbed, self.frame) = self.stream.read()
        
#         self.stopped = False
        
#     def start(self):
#         # Start the thread to read frames from the video stream
#         Thread(target=self.update, args=()).start()
#         return self
    
#     def update(self):
#         while True:
#             if self.stopped:
#                 return
            
#             (self.grabbed, self.frame) = self.stream.read()
            
#     def read(self):
#         return self.frame
    
#     def stop(self):
#         self.stopped = True  

In [11]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
gpu_num = 1

In [12]:
model_path = '../logs/human_pose_dataset_1400_416_yolo/trained_weights_final.h5'

anchors_path = '../model_data/yolo_anchors.txt'
classes_path = '../model_data/human_pose.txt'
score = 0.7
iou = 0.25
model_image_size = (416, 416)
sess = K.get_session()

In [13]:
# Get class
classes_path = os.path.expanduser(classes_path)
with open(classes_path) as f:
    class_names = f.readlines()

class_names = [c.strip() for c in class_names]

# Anchors
anchors_path = os.path.expanduser(anchors_path)
with open(anchors_path) as f:
    anchors = f.readline()
anchors = [float(x) for x in anchors.split(',')]
anchors = np.array(anchors).reshape(-1, 2)

In [14]:
# Load model
model_path = os.path.expanduser(model_path)
assert model_path.endswith('.h5'), 'Keras model end with file .h5'

num_anchors = len(anchors)
num_classes = len(class_names)

is_tiny_version = num_anchors==6
try:
    yolo_model = load_model(model_path, compile=False)
except:
    if is_tiny_version:
        yolo_model = tiny_yolo_body(Input(shape=(None, None, 3)), num_anchors//2, num_classes)
    else:
        yolo_model = yolo_body(Input(shape=(None, None, 3)), num_anchors//3, num_classes)
    
    yolo_model.load_weights(model_path)
else:
    yolo_model.layers[-1].output_shape[-1] == num_anchors/len(yolo_model.output) * (num_classes + 5), 'Mismatch between model and given anchor and class sizes'
    
print("{} model, anchors, and classes loaded.".format(model_path))

OSError: Unable to open file (unable to open file: name = '../logs/human_pose_dataset_1400_416_yolo/trained_weights_final.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
centerpoints = []
namefromcenterpoint = []

In [ ]:
face_encodings_in_room = []
face_names_in_room = []
known_face_encodings_array = np.load("../data/numpy/known_face_encoding.npy")
known_face_names = np.load("../data/numpy/known_face_names.npy")

# Convert nparray -> List to face_encoding
len_of_array_known_face_names = len(known_face_names)
known_face_encodings_array = known_face_encodings_array.reshape(len_of_array_known_face_names, 128)
known_face_encodings = []
for i in range(len_of_array_known_face_names):
    known_face_encodings.append(known_face_encodings_array[i])

In [ ]:
def compare_points(centerpoints, point):
    distance_centerpoint = [distance.euclidean(centerpoints[i], point) for i in range(len(centerpoints))]  
    index_distance_min = np.argmin(distance_centerpoint)
    distance_min = np.min(distance_centerpoint)
    return index_distance_min, distance_min

In [ ]:
# Use face recognition
def detect_name(frame, face_locations, face_encodings, known_face_encodings, box, label):
    top, left, bottom, right = box
    for (top1, right1, bottom1, left1), face_encoding in zip(face_locations, face_encodings):
        distance = face_recognition.face_distance(known_face_encodings, face_encoding)
        min_distance = np.min(distance)
        index_point_min = np.argmin(distance)
        if min_distance < 0.5:
            name = known_face_names[index_point_min]
            print(name)
#                     cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 3)
            label = name + ": " + label
            cv2.putText(frame, label, (left + 6, top + 20), font, 1.0, (0, 0, 255), 1)
        else:
            label = "unknown" + ": " + label
            cv2.putText(frame, label, (left + 6, top + 20), font, 1.0, (0, 0, 255), 1)
    return frame, name

In [ ]:
input_image_shape = K.placeholder(shape=(2, ))
boxes, scores, classes = yolo_eval(yolo_model.output, anchors, len(class_names), input_image_shape, score_threshold=score, iou_threshold=iou)
num_frame = 0
font = cv2.FONT_HERSHEY_DUPLEX

centerpoints = []
namefromcenterpoint = []

# Video capture
video_capture = WebcamVideoStream(src=0).start()

while True:
    num_frame += 1

    # Read video frame and flip camera
    frame = video_capture.read()
    frame = cv2.flip(frame, 1)
    frame_process = np.copy(frame)
        
    # Detect state standing and sleeping and sitting
    image = Image.fromarray(frame_process)

    # Process detect hand and recognition furniture
    boxed_image = letterbox_image(image, tuple(reversed(model_image_size)))
    image_data = np.array(boxed_image, dtype='float32')
    
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)
    
    # Rim keras backend tensorflow forward neural network
    out_boxes, out_scores, out_classes = sess.run([boxes, scores, classes],
                                                 feed_dict={
                                                     yolo_model.input: image_data,
                                                     input_image_shape: [image.size[1], image.size[0]],
                                                     K.learning_phase(): 0
                                                 })

    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]
        
        label = '{} {:.2f}'.format(predicted_class, score)
        
        top, left, bottom, right = box
        print(type(top))
        top = int(top)
        left = int(left)
        bottom = int(bottom)
        right = int(right)
    
        cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 3)        
    
        # Tracking object use center point
        x_point_new = (top + bottom) / 2.
        y_point_new = (left + right) / 2.
    
        point = np.asarray([x_point_new, y_point_new])
        
        if centerpoints:
            index_distance_min, distance_min = compare_points(centerpoints, point)

            # Compare distance min with (bottom - top) / 4
            if distance_min < (bottom - top) / 4.:
                # point new same name index distance min
                name = namefromcenterpoint[index_distance_min]        
                label = name + ": " + label + "don't compute"
                cv2.putText(frame, label, (left + 6, top + 20), font, 1.0, (0, 0, 255), 1)

                # Update center point
                centerpoints[index_distance_min] = point
            else:
                #-------------------------------------------------------#
                # Face recognition
                crop_img = frame_process[top:bottom, left:right]
                # Convert the image from BGR color to RGB to face_recognition use
                rgb_frame = crop_img[:, :, ::-1]

                # Find all the faces and face encodings in the current frame of video
                face_locations = face_recognition.face_locations(rgb_frame)
                face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

                if not face_encodings:
                    cv2.putText(frame, label, (left + 6, top + 20), font, 1.0, (0, 0, 255), 1)
                else:
                    frame, name = detect_name(frame, face_locations, face_encodings, known_face_encodings, 
                                known_face_names, (top, left, bottom, right), label)
                    centerpoints.append(point)
                    namefromcenterpoint.append(name)
        else:
            # Face recognition
            crop_img = frame_process[top:bottom, left:right]
            # Convert the image from BGR color to RGB to face_recognition use
            rgb_frame = crop_img[:, :, ::-1]

            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            if not face_encodings:
                cv2.putText(frame, label, (left + 6, top + 20), font, 1.0, (0, 0, 255), 1)
            else:
                frame, name = detect_name(frame, face_locations, face_encodings, known_face_encodings, 
                            known_face_names, (top, left, bottom, right), label)
                centerpoints.append(point)
                namefromcenterpoint.append(name)
    #         #-------------------------------------------------------#        

    cv2.imshow("Frame", frame)
    #
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

## Tracking multi faces

In [1]:
import cv2
import dlib
import face_recognition
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf


In [2]:
def load_data(path):
    face_encodings_in_room = []
    face_names_in_room = []
    known_face_encodings_array = np.load(path + "known_face_encoding.npy")
    known_face_names = np.load(path + "known_face_names.npy")
    
    # Convert nparray -> list to face encoding
    len_face = len(known_face_names)
    known_face_encodings_array = known_face_encodings_array.reshape(len_face, 128)
    known_face_encodings_list = []
    for i in range(len_face):
        known_face_encodings_list.append(known_face_encodings_array[i])
    
    return known_face_encodings_list, known_face_names

In [3]:
from threading import Thread
import cv2
import imutils
class WebcamVideoStream:
    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        self.stream.set(3, 800)
        self.stream.set(4, 600)
        (self.grabbed, self.frame) = self.stream.read()
        
        self.stopped = False
        
    def start(self):
        # Start the thread to read frames from the video stream
        Thread(target=self.update, args=()).start()
        return self
    
    def update(self):
        while True:
            if self.stopped:
                return
            
            (self.grabbed, self.frame) = self.stream.read()
            
    def read(self):
        return self.frame
    
    def stop(self):
        self.stopped = True

In [4]:
known_face_ecodings_list, known_face_names = load_data("../data/numpy/")

In [5]:
tracker = cv2.MultiTracker_create()

In [6]:
dir(tracker)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'add',
 'clear',
 'create',
 'empty',
 'getDefaultName',
 'getObjects',
 'read',
 'save',
 'update',
 'write']

In [7]:
def computer_area(temp_box, list_box):
    dx = min(a.xmax,)
    

In [ ]:
video_capture = WebcamVideoStream(src=0).start()
number_frame = 0
font = cv2.FONT_HERSHEY_SIMPLEX
tracker = cv2.MultiTracker_create()
init_once = False
name_boxes = []
boxes_known = []
name_known = []
while True:
    frame = video_capture.read()
#     rgb_frame = frame[:, :, ::-1]
    frame_process = frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    
    if (number_frame % 10 == 0):
        # Delete tracker and initi tracker
        tracker.clear()
        tracker = cv2.MultiTracker_create()
#         name_boxes = []
        if init_once == True:
            # Create tracker with name known from before frame, and init name_boxes with people known
            #
            name_boxes_temp = [] # name of people 
            boxes_known = [] # box of people known
            print("name boxes 1", name_boxes)
            for newbow, name in zip(boxes, name_boxes):
                if name != "unknown":
                    ok = tracker.add(cv2.TrackerMIL_create(), frame, (newbow[0], newbow[1] , newbow[2], newbow[3]))
                    name_boxes_temp.append(name)
                    boxes_known.append(newbow)
            print(name_boxes_temp)
        # With people known, continue face recognition in next frame
        # And only change name unknown if (area between unknown and predict > 90 %)
            name_boxes = name_boxes_temp
            print("name boxes 2", name_boxes)
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            if len(known_face_ecodings_list) > 0:
                distance = face_recognition.face_distance(known_face_ecodings_list, face_encoding)
                point = np.min(distance)
                index_point_min = np.argmin(distance)
#                 print(top, right, bottom, left)
                # left top, 
#                 ok = tracker.add(cv2.TrackerMIL_create(), frame, (139, 510, 325, 324))
                temp_box = (top, left, bottom, right)
                if point > 0.4:
                    name = "Unknown"
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)
                    cv2.putText(frame, name, (left + 6, top - 6), font, 1.0, (0, 0, 255), 1)
                elif point <= 0.4:
                    name = known_face_names[index_point_min]
                    print(name)
                    print("name_boxes 3", name_boxes)
                    if name in not name_boxes:
                        ok = tracker.add(cv2.TrackerMIL_create(), frame, (left, top, (right-left), (bottom-top)))
                        cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 3)
                        cv2.putText(frame, name, (left + 6, top - 6), font, 1.0, (0, 0, 255), 1)
#                     boxes_known.append((left, top, (right-left), ()))
                        name_boxes.append(name)
                        print("name_boxes 4", name_boxes)
                    print("name_boxes 5", name_boxes)
    ok, boxes = tracker.update(frame)
    print(tracker)
    print(boxes)
    init_once = True
    for newbox, name in zip(boxes, name_boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, p1, p2, (0, 0, 255))
        cv2.putText(frame, name, p1, font, 1.0, (0, 0, 255), 1)
    cv2.imshow("frame", frame)
               
    number_frame += 1
               
    # Press Q on keyboard to stop recording
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
name boxes 1 []
[]
name boxes 2 []
<MultiTracker 0x7fc8647c3fd0>
()
<MultiTracker 0x7fc8647c3fd0>
()
<MultiTracker 0x7fc8647c3fd0>
()
<MultiTracker 0x7fc8647c3fd0>
()
<MultiTracker 0x7fc8647c3fd0>
()
<MultiTracker 0x7fc8647c3fd0>
()
<MultiTracker 0x7fc8647c3fd0>
()
<MultiTracker 0x7fc8647c3fd0>
()
<MultiTracker 0x7fc8647c3fd0>
()
<MultiTracker 0x7fc8647c3fd0>
()
name boxes 1 []
[]
name boxes 2 []
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<MultiTracker 0x7fc8647c3f90>
()
<Multi

<MultiTracker 0x7fc86461d0f0>
[[422. 217. 186. 186.]
 [410. 226. 185. 186.]
 [371. 154. 223. 223.]
 [438.  88. 223. 223.]
 [344. 159. 223. 223.]]
<MultiTracker 0x7fc86461d0f0>
[[441. 201. 186. 186.]
 [430. 226. 185. 186.]
 [380. 131. 223. 223.]
 [446.  65. 223. 223.]
 [362. 142. 223. 223.]]


In [ ]:
type(box)

In [18]:
a = [1, 2, 3, 4, 5, 6, 6]

In [20]:
b = a

In [22]:
for i in a:
    if i == 6:
        b.remove(i)
        print(b)

[1, 2, 3, 4, 5]
